In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import numpy as np
import pandas as pd
from itertools import product
import numpy as np
import pandas as pd
import os
import openai
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

In [2]:
import sys
import numpy.core
import pandas as pd

sys.modules['numpy._core'] = numpy.core
sys.modules['numpy._core.numeric'] = numpy.core.numeric
sys.modules['numpy._core.multiarray'] = numpy.core.multiarray

df = pd.read_pickle("employee_tasks_hybrid.pkl")
df.head(10)

,task_description,task_embedding,role,employee_id
0,Promote the promotional video to maximize ROI.,"[-0.0052972673, -0.040276837, -0.060003854, -0...",marketing_specialist,4
1,Preprocess the predictive algorithm for the st...,"[-0.0146199, 0.035569996, -0.034567244, -0.001...",data_scientist,2
2,Architect the load balancer to improve scalabi...,"[-0.09025291, 0.039642435, -0.048329785, -0.00...",backend_dev,0
3,Cluster the churn prediction model to improve ...,"[-0.022002958, -0.09358052, 0.002067702, 0.016...",data_scientist,2
4,Update the legacy backend's documentation on a...,"[-0.067919455, -0.020113744, 0.023183897, -0.0...",backend_dev,0
5,Fine-tune a pre-trained BERT model for sentime...,"[-0.089952365, -0.07906764, 0.05981445, 0.0610...",data_scientist,2
6,Escalate the account lockout to avoid negative...,"[-0.03948977, 0.036941875, -0.031007854, -0.01...",customer_support,1
7,Walk through the bug report as a high priority.,"[-0.07357864, -0.0009417966, -0.008004949, 0.0...",customer_support,1
8,Monitor the influencer partnership to improve ...,"[0.029899083, -0.005309763, -0.10674472, -0.04...",marketing_specialist,4
9,Create the influencer partnership with the des...,"[-0.067779005, 0.043094087, -0.031731308, -0.0...",marketing_specialist,4


In [3]:
PARTICIPANTS_PER_MEETING = 5

# Trim to a multiple of 5 so every meeting is complete
n_usable = (len(df) // PARTICIPANTS_PER_MEETING) * PARTICIPANTS_PER_MEETING
df_trimmed = df.iloc[:n_usable].reset_index(drop=True)

# Aggiungi meeting_id al dataset originale (ogni 5 righe consecutive = 1 meeting)
df_trimmed["meeting_id"] = df_trimmed.index // PARTICIPANTS_PER_MEETING

# Assign a meeting_id to each group of 5
# Position within the meeting (1-based): 1st row -> employee_id_1, 2nd -> employee_id_2, ...
df_trimmed["speaker_index_in_meeting"] = (df_trimmed.index % PARTICIPANTS_PER_MEETING) + 1

rows = []
for meeting_id, group in df_trimmed.groupby("meeting_id"):
    row_dict = {"meeting_id": meeting_id}
    for _, r in group.iterrows():
        pos = int(r["speaker_index_in_meeting"])
        row_dict[f"employee_id_{pos}"] = r["employee_id"]
        row_dict[f"task_{pos}"]        = r["task_description"]
        row_dict[f"embedding_{pos}"]   = r["task_embedding"]
        row_dict[f"role_{pos}"]        = r["role"]
    rows.append(row_dict)

df_meetings = pd.DataFrame(rows)

# Reorder columns nicely
col_order = ["meeting_id"]
for i in range(1, PARTICIPANTS_PER_MEETING + 1):
    col_order += [f"employee_id_{i}", f"task_{i}", f"embedding_{i}", f"role_{i}"]
df_meetings = df_meetings[col_order]

print(f"Created {len(df_meetings)} meetings from {n_usable} task rows")
df_meetings.head()

Created 350 meetings from 1750 task rows


,meeting_id,employee_id_1,task_1,embedding_1,role_1,employee_id_2,task_2,embedding_2,role_2,employee_id_3,...,embedding_3,role_3,employee_id_4,task_4,embedding_4,role_4,employee_id_5,task_5,embedding_5,role_5
0,0,4,Promote the promotional video to maximize ROI.,"[-0.0052972673, -0.040276837, -0.060003854, -0...",marketing_specialist,2,Preprocess the predictive algorithm for the st...,"[-0.0146199, 0.035569996, -0.034567244, -0.001...",data_scientist,0,...,"[-0.09025291, 0.039642435, -0.048329785, -0.00...",backend_dev,2,Cluster the churn prediction model to improve ...,"[-0.022002958, -0.09358052, 0.002067702, 0.016...",data_scientist,0,Update the legacy backend's documentation on a...,"[-0.067919455, -0.020113744, 0.023183897, -0.0...",backend_dev
1,1,2,Fine-tune a pre-trained BERT model for sentime...,"[-0.089952365, -0.07906764, 0.05981445, 0.0610...",data_scientist,1,Escalate the account lockout to avoid negative...,"[-0.03948977, 0.036941875, -0.031007854, -0.01...",customer_support,1,...,"[-0.07357864, -0.0009417966, -0.008004949, 0.0...",customer_support,4,Monitor the influencer partnership to improve ...,"[0.029899083, -0.005309763, -0.10674472, -0.04...",marketing_specialist,4,Create the influencer partnership with the des...,"[-0.067779005, 0.043094087, -0.031731308, -0.0...",marketing_specialist
2,2,2,Cluster the exploratory analysis to support th...,"[0.037050847, 0.0041919257, -0.0576101, -0.021...",data_scientist,3,Screen the training workshop following labor l...,"[-0.046370238, 0.016683552, -0.058178768, 0.01...",hr_specialist,3,...,"[0.04611061, 0.016853645, -0.04381959, 0.04400...",hr_specialist,2,Secure the prediction API with an API key and ...,"[-0.13670999, 0.050155967, -0.06901886, 0.0332...",data_scientist,3,Approve the team building event for the annual...,"[0.027130743, 0.08076308, 0.059049938, 0.04405...",hr_specialist
3,3,0,Migrate the RESTful interface using Docker con...,"[0.0037242007, 0.02277143, 0.018757148, -0.039...",backend_dev,0,Test the authentication service to prevent SQL...,"[-0.032426536, -0.026387846, -0.091118865, 0.0...",backend_dev,1,...,"[-0.012072565, 0.013224984, 0.037454497, 0.006...",customer_support,0,Migrate the CI/CD pipeline to handle high traf...,"[0.041368887, -0.004778588, 0.026109176, -0.05...",backend_dev,0,Handle multipart/form-data for high-speed uplo...,"[-0.013614643, 0.017300472, 0.019414539, -0.03...",backend_dev
4,4,0,Refactor the message queue before the code fre...,"[-0.010272045, -0.008613122, -0.020407794, 0.0...",backend_dev,2,Evaluate the exploratory analysis using Python...,"[0.030685244, 0.0895661, -0.11366285, 0.002799...",data_scientist,3,...,"[0.016669387, 0.099373534, 0.0153375575, -0.01...",hr_specialist,1,Escalate the duplicate ticket as a high priority.,"[0.012511725, -0.017118096, 0.057956066, -0.08...",customer_support,3,Survey the training workshop before the hiring...,"[-0.0616273, -0.030740501, -0.00912432, 0.0761...",hr_specialist


In [4]:
del df

In [5]:
df_trimmed.head()

,task_description,task_embedding,role,employee_id,meeting_id,speaker_index_in_meeting
0,Promote the promotional video to maximize ROI.,"[-0.0052972673, -0.040276837, -0.060003854, -0...",marketing_specialist,4,0,1
1,Preprocess the predictive algorithm for the st...,"[-0.0146199, 0.035569996, -0.034567244, -0.001...",data_scientist,2,0,2
2,Architect the load balancer to improve scalabi...,"[-0.09025291, 0.039642435, -0.048329785, -0.00...",backend_dev,0,0,3
3,Cluster the churn prediction model to improve ...,"[-0.022002958, -0.09358052, 0.002067702, 0.016...",data_scientist,2,0,4
4,Update the legacy backend's documentation on a...,"[-0.067919455, -0.020113744, 0.023183897, -0.0...",backend_dev,0,0,5


In [6]:
df_trimmed.to_pickle("employee_tasks_hybrid_meetings_id.pkl")

In [7]:
for var_name in sorted(dir()):
    if not var_name.startswith("_"):
        obj = eval(var_name)
        print(f"{var_name}: {type(obj).__name__}")

In: list
Out: dict
PARTICIPANTS_PER_MEETING: int
PCA: ABCMeta
col_order: list
df_meetings: DataFrame
df_trimmed: DataFrame
exit: ZMQExitAutocall
get_ipython: method
group: DataFrame
i: int
meeting_id: int
n_usable: int
np: module
numpy: module
open: function
openai: module
os: module
pd: module
pdist: function
plt: module
pos: int
product: type
quit: ZMQExitAutocall
r: Series
row_dict: dict
rows: list
squareform: function
sys: module
tqdm: type


In [8]:
os.environ["OPENAI_API_KEY"] = 'sk-MUW96idC6Mker23Qmy09Bg'

client = openai.OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],  
    base_url="https://litellm.sph-prod.ethz.ch/v1",
)

In [9]:
def build_prompt_from_row(row, max_participants=5):
    """Build a prompt using only the participants present in the row.

    Speakers are named generically as Employee 1, Employee 2, ...
    but each Employee i corresponds to a specific underlying role
    (analyst, manager, engineer, etc.) as defined in the dataset.
    """
    lines = []
    speakers = []

    # Prefer explicit count if available, otherwise infer from non-null employee_id_* columns
    n_participants = row.get("n_participants")
    try:
        n_participants = int(n_participants) if pd.notna(n_participants) else None
    except Exception:
        n_participants = None

    if n_participants is None:
        n_participants = 0
        for i in range(1, max_participants + 1):
            if pd.notna(row.get(f"employee_id_{i}")):
                n_participants += 1

    for i in range(1, n_participants + 1):
        emp_id = row.get(f"employee_id_{i}")
        role = row.get(f"role_{i}")  # not shown in speaker label, but kept for semantics
        task = row.get(f"task_{i}")
        if pd.isna(emp_id) or pd.isna(task):
            continue

        speaker = f"Speaker {i}"
        speakers.append(speaker)

        # role_str = str(role).replace("_", " ").title() if pd.notna(role) else None
        # desc_prefix = f"({role_str}) " if role_str else ""
        # lines.append(f"{speaker}: {desc_prefix}{task}")

        lines.append(f"{speaker}: {task}")

    tasks_block = "\n".join(lines)
    unique_speakers = ", ".join(dict.fromkeys(speakers))


    prompt = f"""You are generating a realistic short meeting transcript.
Participants (use only these speaker names): {unique_speakers} (Do NOT replace speaker names with real names but Use exactly Speaker_i with i from 1 to 5.)
Do NOT replace these labels with real human names. Use EXACTLY: {unique_speakers}.
Each participant is working on these work tasks:
Each speaker has a specific work task they are currently working on:
{tasks_block}

Instructions:
- Start the transcript with the line "Meeting."
- Use ONLY the speakers listed above. No sensitive or personal content.
- This is a generic office meeting; write in a neutral, professional tone."""
    #print(prompt)
    return prompt.strip()

In [10]:
def _get_speakers_from_row(row, max_n=5):
    speakers = []
    for i in range(1, max_n + 1):
        if pd.notna(row.get(f"employee_id_{i}")):
            speakers.append(f"Employee {i}")
    return ", ".join(speakers)

def _extract_text(response):
    msg = response.choices[0].message
    content = getattr(msg, "content", None)
    if not content:
        return None
    if isinstance(content, list):
        return "".join(
            (part.get("text", "") if isinstance(part, dict) else str(part))
            for part in content
        ).strip()
    return str(content).strip()

transcripts = []

for _, row in tqdm(df_meetings.iterrows(),total=len(df_meetings)):
    prompt = build_prompt_from_row(row)
    text = None

    for attempt in range(3):
        try:
            response = client.chat.completions.create(
                model="anthropic/claude-sonnet-4-5",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=400,
            )
            content = getattr(response.choices[0].message, "content", None)
            if content:
                text = _extract_text(response)
                if text:
                    break
        except Exception as e:
            print(f"[ERROR] meeting_id={row.get('meeting_id')} tentativo {attempt+1}: {e}")

    # Se refusal o nessun contenuto: retry con prompt minimo (solo dialogo generico, nessun task)
    if not text:
        speakers = _get_speakers_from_row(row)
        fallback = f"""Write a short, neutral work meeting dialogue. Speakers: {speakers}. Start with "Meeting." Then write 12-15 sentences of generic professional dialogue. Use only these speakers."""
        for _ in range(2):
            try:
                response = client.chat.completions.create(
                    model="anthropic/claude-sonnet-4-5",
                    messages=[{"role": "user", "content": fallback}],
                    temperature=0.7,
                    max_tokens=350,
                )
                text = _extract_text(response)
                if text:
                    break
            except Exception as e:
                print(f"[ERROR] fallback meeting_id={row.get('meeting_id')}: {e}")

    transcripts.append(text or "")

# Aggiungi la colonna solo alle prime 5 righe
#df_meetings.loc[df_meetings.index[:5], "meeting_transcript"] = transcripts
df_meetings["meeting_transcript"] = transcripts
df_meetings.head(5)

  0%|          | 0/350 [00:00<?, ?it/s]

,meeting_id,employee_id_1,task_1,embedding_1,role_1,employee_id_2,task_2,embedding_2,role_2,employee_id_3,...,role_3,employee_id_4,task_4,embedding_4,role_4,employee_id_5,task_5,embedding_5,role_5,meeting_transcript
0,0,4,Promote the promotional video to maximize ROI.,"[-0.0052972673, -0.040276837, -0.060003854, -0...",marketing_specialist,2,Preprocess the predictive algorithm for the st...,"[-0.0146199, 0.035569996, -0.034567244, -0.001...",data_scientist,0,...,backend_dev,2,Cluster the churn prediction model to improve ...,"[-0.022002958, -0.09358052, 0.002067702, 0.016...",data_scientist,0,Update the legacy backend's documentation on a...,"[-0.067919455, -0.020113744, 0.023183897, -0.0...",backend_dev,Meeting.\n\n**Speaker 1:** Good morning everyo...
1,1,2,Fine-tune a pre-trained BERT model for sentime...,"[-0.089952365, -0.07906764, 0.05981445, 0.0610...",data_scientist,1,Escalate the account lockout to avoid negative...,"[-0.03948977, 0.036941875, -0.031007854, -0.01...",customer_support,1,...,customer_support,4,Monitor the influencer partnership to improve ...,"[0.029899083, -0.005309763, -0.10674472, -0.04...",marketing_specialist,4,Create the influencer partnership with the des...,"[-0.067779005, 0.043094087, -0.031731308, -0.0...",marketing_specialist,Meeting.\n\n**Speaker 1:** Good morning everyo...
2,2,2,Cluster the exploratory analysis to support th...,"[0.037050847, 0.0041919257, -0.0576101, -0.021...",data_scientist,3,Screen the training workshop following labor l...,"[-0.046370238, 0.016683552, -0.058178768, 0.01...",hr_specialist,3,...,hr_specialist,2,Secure the prediction API with an API key and ...,"[-0.13670999, 0.050155967, -0.06901886, 0.0332...",data_scientist,3,Approve the team building event for the annual...,"[0.027130743, 0.08076308, 0.059049938, 0.04405...",hr_specialist,"Meeting.\n\n**Speaker 1:** Good morning, every..."
3,3,0,Migrate the RESTful interface using Docker con...,"[0.0037242007, 0.02277143, 0.018757148, -0.039...",backend_dev,0,Test the authentication service to prevent SQL...,"[-0.032426536, -0.026387846, -0.091118865, 0.0...",backend_dev,1,...,customer_support,0,Migrate the CI/CD pipeline to handle high traf...,"[0.041368887, -0.004778588, 0.026109176, -0.05...",backend_dev,0,Handle multipart/form-data for high-speed uplo...,"[-0.013614643, 0.017300472, 0.019414539, -0.03...",backend_dev,"Meeting.\n\n**Speaker 1:** Good morning, every..."
4,4,0,Refactor the message queue before the code fre...,"[-0.010272045, -0.008613122, -0.020407794, 0.0...",backend_dev,2,Evaluate the exploratory analysis using Python...,"[0.030685244, 0.0895661, -0.11366285, 0.002799...",data_scientist,3,...,hr_specialist,1,Escalate the duplicate ticket as a high priority.,"[0.012511725, -0.017118096, 0.057956066, -0.08...",customer_support,3,Survey the training workshop before the hiring...,"[-0.0616273, -0.030740501, -0.00912432, 0.0761...",hr_specialist,"Meeting.\n\n**Speaker 1:** Good morning, every..."


In [11]:
# Create a LONG dataset: one row per (meeting, employee-slot)
# Columns: meeting_id, employee_id, task_description, role, task_embedding, meeting_transcript
df_wide = df_meetings.copy()
MAX_PARTICIPANTS = 5

long_parts = []
for i in range(1, MAX_PARTICIPANTS + 1):
    cols = [
        "meeting_id",
        "meeting_transcript",
        f"employee_id_{i}",
        f"task_{i}",
        f"role_{i}",
        f"embedding_{i}",
    ]

    cols_existing = [c for c in cols if c in df_wide.columns]
    tmp = df_wide[cols_existing].copy()

    rename_map = {
        f"employee_id_{i}": "employee_id",
        f"task_{i}": "task_description",
        f"role_{i}": "role",
        f"embedding_{i}": "task_embedding",
    }
    tmp = tmp.rename(columns=rename_map)

    if "role" not in tmp.columns:
        tmp["role"] = pd.NA
    if "task_embedding" not in tmp.columns:
        tmp["task_embedding"] = pd.NA

    tmp = tmp[tmp["employee_id"].notna() & tmp["task_description"].notna()]

    long_parts.append(tmp)

df_long = (
    pd.concat(long_parts, ignore_index=True)
    [["meeting_id", "task_description", "task_embedding", "meeting_transcript", "role", "employee_id"]]
    .sort_values(["meeting_id", "employee_id"], kind="stable")
    .reset_index(drop=True)
)

# Remove rows with no transcript
df_long = df_long.dropna(subset=["meeting_transcript"])
df_long = df_long[df_long["meeting_transcript"].astype(str).str.strip() != ""]

df_long.head()

,meeting_id,task_description,task_embedding,meeting_transcript,role,employee_id
0,0,Architect the load balancer to improve scalabi...,"[-0.09025291, 0.039642435, -0.048329785, -0.00...",Meeting.\n\n**Speaker 1:** Good morning everyo...,backend_dev,0
1,0,Update the legacy backend's documentation on a...,"[-0.067919455, -0.020113744, 0.023183897, -0.0...",Meeting.\n\n**Speaker 1:** Good morning everyo...,backend_dev,0
2,0,Preprocess the predictive algorithm for the st...,"[-0.0146199, 0.035569996, -0.034567244, -0.001...",Meeting.\n\n**Speaker 1:** Good morning everyo...,data_scientist,2
3,0,Cluster the churn prediction model to improve ...,"[-0.022002958, -0.09358052, 0.002067702, 0.016...",Meeting.\n\n**Speaker 1:** Good morning everyo...,data_scientist,2
4,0,Promote the promotional video to maximize ROI.,"[-0.0052972673, -0.040276837, -0.060003854, -0...",Meeting.\n\n**Speaker 1:** Good morning everyo...,marketing_specialist,4


In [12]:
df_meetings.head(10)

# Dataset con solo meeting_id e transcript (uno per meeting)
df_long_transcripts = (
    df_meetings[["meeting_id", "meeting_transcript"]]
    .drop_duplicates(subset="meeting_id", keep="first")
    .reset_index(drop=True)
)

# Dataset con tutte le altre colonne, senza transcript
df_long_details = df_meetings.drop(columns=["meeting_transcript"]).reset_index(drop=True)

print("Transcripts:", df_long_transcripts.shape)
print("Details:", df_long_details.shape)

Transcripts: (350, 2)
Details: (350, 21)


In [ ]:
df_long_transcripts.head(5)

,meeting_id,meeting_transcript
0,0,Meeting.\n\n**Speaker 1:** Good morning everyo...
1,1,Meeting.\n\n**Speaker 1:** Good morning everyo...
2,2,"Meeting.\n\n**Speaker 1:** Good morning, every..."
3,3,"Meeting.\n\n**Speaker 1:** Good morning, every..."
4,4,"Meeting.\n\n**Speaker 1:** Good morning, every..."


In [17]:
df_long_transcripts.to_csv("final_transcripts_meetings.csv", index=False)

In [18]:
df_long = pd.read_csv("final_transcripts_meetings.csv")
df_long.head()

,meeting_id,meeting_transcript
0,0,Meeting.\n\n**Speaker 1:** Good morning everyo...
1,1,Meeting.\n\n**Speaker 1:** Good morning everyo...
2,2,"Meeting.\n\n**Speaker 1:** Good morning, every..."
3,3,"Meeting.\n\n**Speaker 1:** Good morning, every..."
4,4,"Meeting.\n\n**Speaker 1:** Good morning, every..."


In [19]:
# Transcript of the row with meeting_id = 5 (same transcript for all rows of that meeting)
transcript = df_long.loc[df_long["meeting_id"] == 15, "meeting_transcript"].iloc[0]
print(transcript)

Meeting.

**Speaker 1:** Good morning, everyone. Let's get started with our quick status updates. I'll go first. I've been working on optimizing our API latency. I'm implementing Gzip and Brotli compression on the edge, and initial tests are showing promising results. Should have the final deployment ready by end of week.

**Speaker 2:** That sounds great. On my end, I'm monitoring the case study for the Q4 strategy. We've got some good data coming in, and I'm tracking the metrics daily. I'll have a preliminary report ready for the leadership team by Wednesday.

**Speaker 3:** Thanks. I'm dealing with some login issues for our new hire's accounts. There was a provisioning delay with the SSO system, but IT is on it. I'm expecting everything to be resolved by this afternoon so they can get fully onboarded.

**Speaker 4:** Good to know. I've been working through the helpdesk queue. There was a service outage complaint that needed to be merged with an existing ticket. I've consolidated eve